In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torchvision

In [2]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.disc = nn.Sequential(nn.Linear(img_dim, 128),
                                 nn.LeakyReLU(0.1),
                                 nn.Linear(128, 1),
                                 nn.Sigmoid())
    def forward(self, x):
        return self.disc(x)
        
        
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(nn.Linear(z_dim, 256),
                                nn.LeakyReLU(0.1),
                                nn.Linear(256, img_dim),
                                nn.Tanh())
    def forward(self, x):
        return self.gen(x)

**HyperParameters**

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
learning_rate = 3e-4
z_dim = 64
img_dim = 28*28*1
batch_size = 32
num_epochs = 50

In [4]:
disc = Discriminator(img_dim).to(device)
gen = Generator(z_dim, img_dim).to(device)

In [5]:
fixed_noise = torch.randn(batch_size, z_dim).to(device)

In [6]:
transforms = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),])

In [7]:
dataset = datasets.MNIST(root=r"C:\Users\sankalp\Desktop\pytorch\Data", transform=transforms, download=True)
loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

C:\Users\sankalp\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
loss_function = nn.BCELoss()
opt_disc = optim.Adam(disc.parameters(), lr=learning_rate)
opt_gen = optim.Adam(gen.parameters(), lr=learning_rate)

In [11]:
writer_fake = SummaryWriter(f"logs1/fake")
writer_real = SummaryWriter(f"logs1/real")
step = 0

In [12]:
for epoch in range(num_epochs):
    for batch_index, data in enumerate(loader):
        real, _ = data
        real = real.view(-1,28*28*1).to(device)
        batch_size = real.shape[0]
        
        #Training Discriminator   max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = loss_function(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = loss_function(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_fake + lossD_real) / 2
        
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()
        
        #Trainning Generator   min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).view(-1)
        lossG = loss_function(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        
        if batch_index == 0:
            print(f'Epoch[{epoch}/{num_epochs}], LossG:{lossG:4f}')
            
            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

Epoch[0/50], LossG:2.140658
Epoch[1/50], LossG:0.739536
Epoch[2/50], LossG:0.843695
Epoch[3/50], LossG:0.751847
Epoch[4/50], LossG:1.183138


KeyboardInterrupt: 